## law_1.docx, law_2.docx 문서 두개
- pinecone store 저장 (하나의 인덱스에 저장)
    - index name 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기


In [3]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


##환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

## 문서 읽고 분할 #############################################
## 문서 파일 목록
doc_paths = ['law_1.docx', 'law_2.docx']

## 문서 로드 (읽어오기)
documents = []

for path in doc_paths:
    loader = Docx2txtLoader(path)
    documents.extend(loader.load())

## 청크 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

## 문서 분할 
document_list = text_splitter.split_documents(documents)
len(document_list)

29

In [10]:

## 임베딩 - > 벡터 스토어 (데이터베이스)에 저장

## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'laws'

## 파인콘 : 저장
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='laws',
# )

#저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index('law1'),
#     embedding=embedding,
# )

## [방법 2]
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)


In [ ]:


## RetrievalQA
llm = ChatOpenAI(model='gpt-4o')

prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {'context': database.as_retriever() | format_docs,
     'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기 피해자 대상을 알려주세요.')
qa_chain.invoke('전세사기 피해자 임대인을 알려주세요.')

'전세사기 피해자의 임대인은 전세사기 피해 방지법에 따라 정의됩니다. 임대인 또는 임대인의 대리인, 공인중개사, 임차인을 모집하는 자, 다수 임대인의 배후에 있는 동일인 또는 이와 관련된 조직 등이 해당될 수 있습니다. 이들은 전세사기와 관련하여 임대차 계약에 관여하는 사람들로 정의됩니다.'